# 13.3 项目实施

## 13.3.1 获取所有Live

In [1]:
import requests

def scrapy(link):
    headers = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.98 Safari/537.36'
    } 
    r = requests.get(link, headers= headers)
    return (r.text)

link = "https://api.zhihu.com/lives/homefeed?includes=live"
html = scrapy(link)
print (html)

{"error": {"message": "\u672a\u8bbe\u7f6e\u9a8c\u8bc1\u65b9\u5f0f", "code": 602}}


In [6]:
import requests

def scrapy(link):
    headers = {
        'Host' : 'api.zhihu.com',
        'Origin' : 'https://www.zhihu.com',
        'Referer' : 'https://www.zhihu.com/lives',
        'authorization' : 'oauth 8274ffb553d511e6a7fdacbc328e205d',
        'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.98 Safari/537.36'
    } 
    r = requests.get(link, headers= headers)
    return (r.text)

link = "https://api.zhihu.com/lives/homefeed?includes=live"
html = scrapy(link)
print (html)

{"paging": {"is_end": false, "next": "https://api.zhihu.com/lives/homefeed?limit=10&offset=1494594000", "previous": ""}, "data": [{"object_type": "live", "source_type": "hot_live", "live": {"status": "ended", "ends_at": 1494945036, "liked": false, "ends_in": 0, "speaker_message_count": 99, "tags": [{"score": 0, "name": "体育", "short_name": "体育", "available_num": 136, "created_at": 1469691129, "id": 104, "live_num": 151}], "review": {"count": 9, "status": "normal", "score": 5.0, "has_reviewed": false, "previous": {"status": "nonexistent", "score": 0}}, "liked_num": 103, "live_type": "audio", "is_public": 1, "video": {}, "cospeakers": [{"member": {"name": "胡海清", "url": "https://api.zhihu.com/people/4c29d9b23e17e872ac364fb4d264eebf", "type": "people", "user_type": "people", "headline": "浅石创投", "avatar_url": "https://pic4.zhimg.com/v2-f59d02a3bcb16843677b49be87c03bf7_s.jpg", "gender": 1, "url_token": "hu-hai-qing-33", "id": "4c29d9b23e17e872ac364fb4d264eebf"}, "bio": "浅石创投", "description": 

In [7]:
import json
decodejson = json.loads(html)
next_page = decodejson['paging']['next']
is_end = decodejson['paging']['is_end']
print (next_page)
print (is_end)

https://api.zhihu.com/lives/homefeed?limit=10&offset=1494594000
False


In [9]:
import requests
from pymongo import MongoClient
import json
import time
import random

client = MongoClient('localhost',27017)
db = client.zhihu_database
collection = db.live

def scrapy(link):
    headers = {
        'Host' : 'api.zhihu.com',
        'Origin' : 'https://www.zhihu.com',
        'Referer' : 'https://www.zhihu.com/lives',
        'authorization' : 'oauth 8274ffb553d511e6a7fdacbc328e205d',
        'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.98 Safari/537.36'
    } 
    r = requests.get(link, headers= headers)
    return (r.text)

link = "https://api.zhihu.com/lives/homefeed?includes=live"
is_end = False
while not is_end:
    html = scrapy(link)
    decodejson = json.loads(html)
    collection.insert_one(decodejson)
    
    link = decodejson['paging']['next']
    is_end = decodejson['paging']['is_end']
    time.sleep(random.randint(2,3) + random.random())

## 13.3.2 获取Live的听众

In [24]:
from pymongo import MongoClient
client = MongoClient('localhost',27017)
db = client.zhihu_database
collection = db.live

first_page = collection.find_one()
for each in first_page['data']:
    print (each['live']['id'])

840520148518592512
837669764146003968
837693085466107904
843124614065434624
842731562154987520
839869490635366400
839804209770692608
843110642910117888
840570499422650368
838157855663230976


In [29]:
from pymongo import MongoClient
client = MongoClient('localhost',27017)
db = client.zhihu_database
    
live_id = '840520148518592512'

def get_audience(live_id):
    headers = {
        'Host' : 'api.zhihu.com',
        'Origin' : 'https://www.zhihu.com',
        'Referer' : 'https://www.zhihu.com/lives',
        'authorization' : 'oauth 8274ffb553d511e6a7fdacbc328e205d',
        'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.98 Safari/537.36'
    } 
    link = 'https://api.zhihu.com/lives/' + live_id + '/members?limit=10&offset=0'
    
    is_end = False
    while not is_end:
        r = requests.get(link, headers= headers)
        html = r.text
        decodejson = json.loads(html)
        decodejson['live_id'] = live_id
        db.live_audience.insert_one(decodejson)

        link = decodejson['paging']['next']
        is_end = decodejson['paging']['is_end']
        time.sleep(random.randint(2,3) + random.random())

get_audience(live_id)

In [30]:
from pymongo import MongoClient
client = MongoClient('localhost',27017)
db = client.zhihu_database

for each_page in db.live.find():
    for each in each_page['data']:
        live_id = each['live']['id']
        print (live_id)        
        get_audience(live_id)

840520148518592512
837669764146003968
837693085466107904
843124614065434624
842731562154987520
839869490635366400
839804209770692608
843110642910117888
840570499422650368
838157855663230976
840886183490912256
840879245080276992
837680104363679744
841344265740255232
842735083944308736
842735925179088896
839875793512456192
839859178712465408
835436065190993920
838163090607591424
841289257443274752
837679382431662080
838159504024682496
831457860721479680
840984548597714944
839864761947652096
833702327444725760
838388999964987392
835139866269474816
842740340824879104
819509716299436032
839869964864348160
837682828706746368
838405704879607808
827151989283635200
835125363007946752
836344346444967936
837373383242043392
835136598269112320
830380491126501376
835121024906432512
824618968198291456
831104976066609152


JSONDecodeError: Expecting value: line 1 column 1 (char 0)